In [27]:
import time
import serial
import tkinter as tk
from tkinter import ttk  
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.neighbors import KNeighborsClassifier

# rest of the code remains unchanged

predict_button = None
labelp = None



#read train values in txt format
ser = serial.Serial('COM12', 9600)
f = open('train.txt', 'a')
def train():
    index=input("enter nuumber") 
    i=1
    while(i<=20):
        time.sleep(1)
        data = ser.readline()
        data_str=str(data)
        
        split_data=data_str[2:-5]
        
        val=split_data.split(',')
        for v in val:
            f.write(v)
            f.write(',')
        f.write(index)
        f.write('\n')
        print(i)
        i=i+1
    f.close()
    
#read test value in txt format
f1=open('test.txt','w')
def test():
    start=time.time()
    j=1
    while(j<=10):
        time.sleep(1)
        data=ser.readline()
        data_str=str(data)
        
        split_data=data_str[2:-5]
        
        val=split_data.split(',')
        for v in val:
            f1.write(v)
            f1.write(',')
        f1.write('\n')   
        print(j)
        j=j+1
    f1.close()
    end=time.time()
    msg1 = "Successfully Data Taken in " + str(end - start) + "' S time"
    labelt.config(text=msg1)
    

# test window
def OpenTestWindow():
    global predict_button, labelp
    root1 = tk.Tk()
    root1.geometry("500x500")
    root1.title("test")

    def test_and_show_predict_button():
        test()  # Call the test function
        predict_button.pack(pady=10)  # Show the "Predict" button after data collection

    button = tk.Button(root1, text="Test", command=test_and_show_predict_button)
    button.pack(pady=10)
    global labelt
    labelt = tk.Label(root1, text="")
    labelt.pack()

    # "Predict" button (initially hidden)
    predict_button = tk.Button(root1, text="Predict", command=predict)
    labelp = tk.Label(root1, text="")
    labelp.pack(pady=20)
    
    root1.mainloop()



#Train window  
def OpenTrainWindow():
    root=tk.Tk()
    root.geometry("500x500")
    root.title("Real Time Siting posture monitoring system")
    label_var = tk.StringVar()
    label_var.set("choose position")
    a=["siting upright", "left lean", "right lean", "forward lean", "backward lean", "left leg up", "right leg up"]
    drop=ttk.Combobox(root,textvariable=label_var,values=a)
    drop.pack()
    button=tk.Button(root,text="Train",font=('arial',18),command=train)
    button.pack(pady=10)
    root.mainloop()

#connect this two window
connect=tk.Tk()
connect.geometry("500x500")
connect.title("welcome")
button1=tk.Button(connect,text="Train",font=('arial',18),command=OpenTrainWindow)
button1.pack()
button2=tk.Button(connect,text="Test",font=('arial',18),command=OpenTestWindow)
button2.pack()
connect.mainloop()

#prediction
def predict():
    #train data read in csv
    df = pd.read_csv("train.txt",header=None)
    data_train = df.iloc[:,0:16]  #store the readings of peizo sensor to data_train
    label_train = df.iloc[:,17:18]  #store the positions in label_train
    
    #read the test data in csv
    df_test=pd.read_csv("test.txt",header=None)
    data_test=df_test.iloc[:,0:16]
    
   # ...
    # make a model and fit the data of data_train and label_train
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(data_train, label_train.values.ravel())  # Use ravel() to convert to 1-dimensional array
    
    # ...
    # predict the label of test data 
    result = model.predict(data_test)
    
    try:
        max_mode = int(np.median(result))
    
        if 1 <= max_mode <= 7:
            labels = ["Sitting upright", "Left lean", "Right lean", "Forward lean", "Backward lean", "Left leg on top", "Right leg on top"]
            p = labels[max_mode - 1]
            print(f"Predicted posture: {p}")
        else:
            p = "NA"
            print("Invalid predicted label.")
    except (IndexError, ValueError, np.AxisError) as e:
        print(f"Error: {e}")
        p = "NA"
    
    labelp.config(text=p)



    


1
2
3
4
5
6
7
8
9
10
Predicted posture: Sitting upright
